<a href="https://colab.research.google.com/github/Vamarinpe/PAE/blob/master/CodRecurso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Por agente

# importa la librería
import pandas as pd
import numpy as np
pd.set_option('display.notebook_repr_html', False)
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
import csv

Datos = pd.read_csv(
    "https://raw.githubusercontent.com/Vamarinpe/PAE/master/Recurso.csv",
    sep = ',',         # separador de campos
    thousands = None,  # separador de miles para números
    decimal = '.',
    index_col=["Fecha","Codigo Agente","Recurso"],
    usecols=["Fecha","Codigo Agente","Recurso","Precio Oferta Ideal"],
    parse_dates=["Fecha"],
    dayfirst=True)


#Datos = pd.read_csv(
    #"https://raw.githubusercontent.com/Vamarinpe/Ensayo2/master/Datos.csv",
    #sep = ',',         # separador de campos
    #thousands = None,  # separador de miles para números
    #decimal = '.',
    #quoting=csv.QUOTE_NONNUMERIC)

#Datos['Fecha'] = Datos.Fecha\
                             #.map(lambda x : datetime.strptime(x, '%Y-%m-%d'))

In [0]:
len(Datos)
Datos.columns
Datos.describe()
Datos.head(60)

In [0]:
Datos.count()

In [0]:
Datos.groupby('Codigo Agente')['Precio Oferta Ideal'].count()
Datos.groupby('Recurso')['Precio Oferta Ideal'].count()
Datos.groupby('Fecha')['Precio Oferta Ideal'].count()

In [0]:
Datos.loc[('2012-09-04')]
Datos.loc[('2012-09-04','EPMG')]
#plt.plot(Datos.loc[('2017-09-05','EPMG')])
#plt.show()

In [0]:
Datos.loc['2012-01-01':'2013-12-31']

In [0]:
Datos.xs('EPMG',level='Codigo Agente')
Datos.xs('GUATAPE',level='Recurso')
#Datos.xs('2012-06-08',level='Fecha')

In [0]:
Datos.iloc[5]

In [0]:
plt.plot(Datos['Precio Oferta Ideal'])

plt.ylabel('Precio $/kWh',fontsize=12)
plt.xlabel('Fecha',fontsize=12)
plt.legend(prop={'size': 12})
plt.suptitle('PRECIOS DE OFERTA POR AGENTE',fontsize=18)
plt.rcParams["figure.figsize"] = (20,8)
plt.show()